In [1]:
!pip install -r requirements.txt

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
  Using cached attrs-19.3.0-py2.py3-none-any.whl (39 kB)
  Using cached certifi-2020.4.5.1-py2.py3-none-any.whl (157 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached codecov-2.0.22-py2.py3-none-any.whl (15 kB)
  Using cached coverage-5.0.4-cp27-cp27m-macosx_10_13_intel.whl (208 kB)
  Using cached idna-2.9-py2.py3-none-any.whl (58 kB)
  Using cached importlib_metadata-1.6.0-py2.py3-none-any.whl (30 kB)
ERROR: Could not find a version that satisfies the requirement more-itertools==8.2.0 (from -r requirements.txt (line 9)) (from versions: 1.0, 1.1, 2.0, 2.

In [2]:
import pandas as pd
import numpy as np

In [3]:
dataset = pd.read_csv('data/Telco-Customer-Churn.train.csv')



### Visualizzo il dataset

In [4]:
dataset

,Churn,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,No,4223-BKEOR,Female,0,No,Yes,21,Yes,No,DSL,...,No,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.8
1,No,6035-RIIOM,Female,0,No,No,54,Yes,Yes,Fiber optic,...,Yes,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45
2,Yes,3797-VTIDR,Male,0,Yes,No,1,No,No phone service,DSL,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45
3,Yes,2568-BRGYX,Male,0,No,No,4,Yes,No,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95
4,No,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,Yes,0684-AOSIH,Male,0,Yes,No,1,Yes,No,Fiber optic,...,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,95.00,95
5630,No,5982-PSMKW,Female,0,Yes,Yes,23,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),91.10,2198.3
5631,No,8044-BGWPI,Male,0,Yes,Yes,12,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Electronic check,21.15,306.05
5632,Yes,7450-NWRTR,Male,1,No,No,12,Yes,Yes,Fiber optic,...,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.45,1200.15


### Vedo le chiavi presenti nel database

In [5]:
for el in dataset.columns:
    print("Colonna:"+el+"",dataset[el].unique())

Colonna:Churn ['No' 'Yes']
Colonna:customerID ['4223-BKEOR' '6035-RIIOM' '3797-VTIDR' ... '8044-BGWPI' '7450-NWRTR'
 '4795-UXVCJ']
Colonna:gender ['Female' 'Male']
Colonna:SeniorCitizen [0 1]
Colonna:Partner ['No' 'Yes']
Colonna:Dependents ['Yes' 'No']
Colonna:tenure [21 54  1  4  0  7 32 72 19 10 45 40 47 36 69 71 35  3 68 42  8 46 12 26
 49 33 31 66 58 13 57  6 59 15 27 34 18  5 39 29  2 63 20 14 56 37 24 52
 43 11 16 50 38 23 55 48 53 70 22 28 44 65 64 60 51  9 25 61 30 17 41 67
 62]
Colonna:PhoneService ['Yes' 'No']
Colonna:MultipleLines ['No' 'Yes' 'No phone service']
Colonna:InternetService ['DSL' 'Fiber optic' 'No']
Colonna:OnlineSecurity ['Yes' 'No' 'No internet service']
Colonna:OnlineBackup ['No' 'Yes' 'No internet service']
Colonna:DeviceProtection ['Yes' 'No' 'No internet service']
Colonna:TechSupport ['No' 'Yes' 'No internet service']
Colonna:StreamingTV ['No' 'Yes' 'No internet service']
Colonna:StreamingMovies ['Yes' 'No' 'No internet service']
Colonna:Contract ['One yea

In [6]:
dataset.TotalCharges.head(20)

0      1336.8
1     5129.45
2       23.45
3      237.95
4            
5      521.35
6      613.95
7     1414.65
8      733.35
9       219.5
10    1474.75
11    3062.45
12      587.7
13    4228.55
14    1875.25
15    1901.25
16     7234.8
17    5705.05
18    1710.15
19     1930.9
Name: TotalCharges, dtype: object

In [7]:
dataset= dataset.replace(' ', np.NaN)

### Vado a buttare le poche righe che presentano dei valori nulli nella colonna "TotalCharges"

In [8]:
dataset = dataset.dropna()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5624 entries, 0 to 5633
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Churn             5624 non-null   object 
 1   customerID        5624 non-null   object 
 2   gender            5624 non-null   object 
 3   SeniorCitizen     5624 non-null   int64  
 4   Partner           5624 non-null   object 
 5   Dependents        5624 non-null   object 
 6   tenure            5624 non-null   int64  
 7   PhoneService      5624 non-null   object 
 8   MultipleLines     5624 non-null   object 
 9   InternetService   5624 non-null   object 
 10  OnlineSecurity    5624 non-null   object 
 11  OnlineBackup      5624 non-null   object 
 12  DeviceProtection  5624 non-null   object 
 13  TechSupport       5624 non-null   object 
 14  StreamingTV       5624 non-null   object 
 15  StreamingMovies   5624 non-null   object 
 16  Contract          5624 non-null   object 


### Per poter lavorare in comodità vado a separare i valori categorici da quelli numerico

In [9]:
import numpy as np

In [10]:
features_numeriche = dataset.select_dtypes(include = np.number)
features_numeriche

,SeniorCitizen,tenure,MonthlyCharges
0,0,21,64.85
1,0,54,97.20
2,0,1,23.45
3,0,4,70.20
5,0,7,69.55
...,...,...,...
5629,0,1,95.00
5630,0,23,91.10
5631,0,12,21.15
5632,1,12,99.45


In [11]:
features_categoriche = dataset.select_dtypes(include = 'object')
features_categoriche

,Churn,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,No,4223-BKEOR,Female,No,Yes,Yes,No,DSL,Yes,No,Yes,No,No,Yes,One year,No,Mailed check,1336.8
1,No,6035-RIIOM,Female,No,No,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),5129.45
2,Yes,3797-VTIDR,Male,Yes,No,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,23.45
3,Yes,2568-BRGYX,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,237.95
5,No,4291-SHSBH,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,521.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,Yes,0684-AOSIH,Male,Yes,No,Yes,No,Fiber optic,Yes,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,95
5630,No,5982-PSMKW,Female,Yes,Yes,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),2198.3
5631,No,8044-BGWPI,Male,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Electronic check,306.05
5632,Yes,7450-NWRTR,Male,No,No,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,1200.15


## Label Encoding

Vado ad utilizzare il label encoder per trasformare i dati categorici del dataset(di tipo 'object') in valori interi.


In [12]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()

for i in features_categoriche:
    features_categoriche[i] = label_encoder.fit_transform(features_categoriche[i])
    
features_categoriche

/Users/federicogaglio/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Churn,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0,2364,0,0,1,1,0,0,2,0,2,0,0,2,1,0,3,491
1,0,3448,0,0,0,1,2,1,0,2,0,0,2,2,2,1,0,3510
2,1,2143,1,1,0,0,1,0,0,0,0,0,0,0,0,1,2,1565
3,1,1424,1,0,0,1,0,1,0,0,0,0,0,0,0,1,2,1626
5,0,2394,1,0,0,1,0,1,0,0,0,0,0,0,0,1,2,3548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,1,375,1,1,0,1,0,1,2,0,0,0,2,2,0,1,2,5230
5630,0,3409,0,1,1,1,2,0,2,2,2,2,2,2,2,1,1,1465
5631,0,4581,1,1,1,1,0,2,1,1,1,1,1,1,0,1,2,2166
5632,1,4238,1,0,0,1,2,1,0,0,2,0,2,2,0,1,2,304


### Vado a ricompattare i dati che precedentemente avevo separato in numerici e categorici, nel dataset intero

In [13]:
fullDatasetNumCat = pd.concat([features_numeriche, features_categoriche], axis = 1)

fullDatasetNumCat

,SeniorCitizen,tenure,MonthlyCharges,Churn,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,...,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0,21,64.85,0,2364,0,0,1,1,0,...,2,0,2,0,0,2,1,0,3,491
1,0,54,97.20,0,3448,0,0,0,1,2,...,0,2,0,0,2,2,2,1,0,3510
2,0,1,23.45,1,2143,1,1,0,0,1,...,0,0,0,0,0,0,0,1,2,1565
3,0,4,70.20,1,1424,1,0,0,1,0,...,0,0,0,0,0,0,0,1,2,1626
5,0,7,69.55,0,2394,1,0,0,1,0,...,0,0,0,0,0,0,0,1,2,3548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,0,1,95.00,1,375,1,1,0,1,0,...,2,0,0,0,2,2,0,1,2,5230
5630,0,23,91.10,0,3409,0,1,1,1,2,...,2,2,2,2,2,2,2,1,1,1465
5631,0,12,21.15,0,4581,1,1,1,1,0,...,1,1,1,1,1,1,0,1,2,2166
5632,1,12,99.45,1,4238,1,0,0,1,2,...,0,0,2,0,2,2,0,1,2,304


### Separo il target dal resto dei dati

In [14]:
y = fullDatasetNumCat.Churn

X = fullDatasetNumCat.drop(columns=['Churn'])

In [15]:
y

0       0
1       0
2       1
3       1
5       0
       ..
5629    1
5630    0
5631    0
5632    1
5633    0
Name: Churn, Length: 5624, dtype: int64

In [16]:
X

,SeniorCitizen,tenure,MonthlyCharges,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0,21,64.85,2364,0,0,1,1,0,0,2,0,2,0,0,2,1,0,3,491
1,0,54,97.20,3448,0,0,0,1,2,1,0,2,0,0,2,2,2,1,0,3510
2,0,1,23.45,2143,1,1,0,0,1,0,0,0,0,0,0,0,0,1,2,1565
3,0,4,70.20,1424,1,0,0,1,0,1,0,0,0,0,0,0,0,1,2,1626
5,0,7,69.55,2394,1,0,0,1,0,1,0,0,0,0,0,0,0,1,2,3548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,0,1,95.00,375,1,1,0,1,0,1,2,0,0,0,2,2,0,1,2,5230
5630,0,23,91.10,3409,0,1,1,1,2,0,2,2,2,2,2,2,2,1,1,1465
5631,0,12,21.15,4581,1,1,1,1,0,2,1,1,1,1,1,1,0,1,2,2166
5632,1,12,99.45,4238,1,0,0,1,2,1,0,0,2,0,2,2,0,1,2,304


In [17]:
X

,SeniorCitizen,tenure,MonthlyCharges,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0,21,64.85,2364,0,0,1,1,0,0,2,0,2,0,0,2,1,0,3,491
1,0,54,97.20,3448,0,0,0,1,2,1,0,2,0,0,2,2,2,1,0,3510
2,0,1,23.45,2143,1,1,0,0,1,0,0,0,0,0,0,0,0,1,2,1565
3,0,4,70.20,1424,1,0,0,1,0,1,0,0,0,0,0,0,0,1,2,1626
5,0,7,69.55,2394,1,0,0,1,0,1,0,0,0,0,0,0,0,1,2,3548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,0,1,95.00,375,1,1,0,1,0,1,2,0,0,0,2,2,0,1,2,5230
5630,0,23,91.10,3409,0,1,1,1,2,0,2,2,2,2,2,2,2,1,1,1465
5631,0,12,21.15,4581,1,1,1,1,0,2,1,1,1,1,1,1,0,1,2,2166
5632,1,12,99.45,4238,1,0,0,1,2,1,0,0,2,0,2,2,0,1,2,304


## Features Selection

Per andare a semplicare il modello ed evitare un eccessivo overfitting vado ad utilizzare, per la scelta di un sottoinsieme dei vari attributi, **SelectFromModel**.

Questo strumento, offerto da sklearn, è di fatto un meta-trasformatore che a seconda dello stimatore che gli viene passato, va a fare una selezione delle features ad-hoc.
In questo caso ho scelto di utilizzare la **RidgeRegression**, che è un ottimo strumento per poter andare a diminuire l'overfitting.


L’idea alla base della Ridge Regression è quella di limitare il valore assoluto dei coefficienti **Wi** definendo come segue la funzione di costo totale (da minimizzare nella fase di training):

>> $$costoRidge = misura del “fit” + misura della grandezza dei coefficienti$$

Con misura del fit, che rappresenta una qualsiasi funzione di costo come per esempio l'**RSS**, mentre la grandezza dei coefficienti, per quanto riguarda la RidgeRegression, è rappresentata dall'utilizzo della **l2 norm**.

La **l2 norm** è il fattore di regolarizzazione rappresentato dalla seguente espressione:
>> $$R(W) = \sum _l (w_l)^2$$

Questo parametro di regolarizzazione va a mettere in quadratura tutti i parametri.Successivamente li vado a sommare.
Con parametri piccoli la quadratura mi fa scendere il costo(perchè se ho valori <1, andando a fare il quadrato questi mi diventano ancora più piccoli), con parametri sopra l'1 andrà ad aumentare(perchè ovviamente il quadrato sarà un valore più grande).

La Ridge può quindi essere riscritta come:
>> $$costoRidge(w) = RSS(w) + \lambda * R(w)$$

Come si può notare, affianco alla nostra **l2 norm**, compare un parametro **lambda**. Questo parametro viene detto di tuning, e sostanzialmente serve a bilanciare quella che è l'espressione sopra descritta. **Lambda** può assumere sostanzialmente vari valori e a seconda di quelli che assume abbiamo che la nostra funzione di costo adotta un comportamento differente. Infatti:

1) se lambda=0, come si può facilmente osservare abbiamo che la nostra funzione di costo ridge diventa sostanzialmente una RSS.

2) se lambda invece tende ad infinito, allora abbiamo che l'intero costo mi tende ad infinito e l'unico modo per minimizzare è quello di avere W=0.

Generalmente per alti valori di lambda abbiamo inoltre un alto bias e una bassa vairianza, mentre per bassi valori il viceversa.

In [18]:
from sklearn.feature_selection import SelectFromModel
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso,Ridge




#sfm=SelectFromModel(LogisticRegression(max_iter=100000, C=1000))
sfm=SelectFromModel(Ridge(max_iter=100000, alpha= 0.001))
sfm.fit(X, y)

sfm.get_support()

selected_Ridge = X.columns[(sfm.get_support())]

selected_Ridge

Index(['SeniorCitizen', 'PhoneService', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'TechSupport', 'Contract', 'PaperlessBilling'],
      dtype='object')

In [19]:
X = X[selected_Ridge]
X

,SeniorCitizen,PhoneService,InternetService,OnlineSecurity,OnlineBackup,TechSupport,Contract,PaperlessBilling
0,0,1,0,2,0,0,1,0
1,0,1,1,0,2,0,2,1
2,0,0,0,0,0,0,0,1
3,0,1,1,0,0,0,0,1
5,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...
5629,0,1,1,2,0,0,0,1
5630,0,1,0,2,2,2,2,1
5631,0,1,2,1,1,1,0,1
5632,1,1,1,0,0,0,0,1


## ADDESTRAMENTO DEL MODELLO

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


tr = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=6)


#fit e prediction
tr.fit(X_train, y_train)
pred = tr.predict(X_valid)
print('Accuracy:',accuracy_score(y_valid, pred))

Accuracy: 0.8
